In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import json

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

In [ ]:
# Configuration

# Dataset paths
NO_DETECTOR_DIR = r"C:\Users\thaim\Videos\AI_LEDS\cropped_no_detector_pictures"
DETECTOR_DIR = r"C:\Users\thaim\Videos\AI_LEDS\cropped_detector_pictures"

# Output paths
OUTPUT_DIR = r"C:\Users\thaim\Videos\AI_LEDS\model_output"
MODEL_NAME = "detector_classifier_mobilenetv2"

# Image settings
IMG_SIZE = 224  # MobileNetV2 standard input size
BATCH_SIZE = 32  # Adjust based on GPU memory (16/32/64)

# Training settings
EPOCHS = 30
LEARNING_RATE = 0.001
TEST_SIZE = 0.1      # 10% for test
VAL_SIZE = 0.1       # 10% for validation (from remaining 90%)

# Data augmentation for detector class (minority class)
AUGMENTATION_ENABLED = True

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Load and prepare dataset

def load_image_paths(directory, label):
    """Load all image paths from directory recursively"""
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    paths = []
    
    dir_path = Path(directory)
    for ext in image_extensions:
        paths.extend(dir_path.rglob(f'*{ext}'))
    
    return [(str(p), label) for p in paths]

print("Loading image paths...")
no_detector_data = load_image_paths(NO_DETECTOR_DIR, 0)  # Label 0 = No detector
detector_data = load_image_paths(DETECTOR_DIR, 1)        # Label 1 = Detector

print(f"No detector images: {len(no_detector_data)}")
print(f"Detector images: {len(detector_data)}")
print(f"Total images: {len(no_detector_data) + len(detector_data)}")
print(f"Class imbalance ratio: {len(no_detector_data) / len(detector_data):.2f}:1")

# Combine and shuffle
all_data = no_detector_data + detector_data
np.random.shuffle(all_data)

# Separate paths and labels
image_paths = [d[0] for d in all_data]
labels = np.array([d[1] for d in all_data])

# Split: first 10% for test, then split remaining into train/val
X_temp, X_test, y_temp, y_test = train_test_split(
    image_paths, labels, test_size=TEST_SIZE, stratify=labels, random_state=42
)

# Split remaining into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=VAL_SIZE/(1-TEST_SIZE), stratify=y_temp, random_state=42
)

print(f"\nDataset split:")
print(f"Train: {len(X_train)} images")
print(f"Validation: {len(X_val)} images")
print(f"Test: {len(X_test)} images")
print(f"\nTrain class distribution:")
print(f"  No detector: {np.sum(y_train == 0)}")
print(f"  Detector: {np.sum(y_train == 1)}")

In [ ]:
# Create data generators with augmentation

# Augmentation for training detector images only (minority class)
train_detector_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
) if AUGMENTATION_ENABLED else ImageDataGenerator(rescale=1./255)

# No augmentation for no_detector class and validation/test
train_no_detector_datagen = ImageDataGenerator(rescale=1./255)
val_test_datagen = ImageDataGenerator(rescale=1./255)

def create_generator(paths, labels, datagen, batch_size, augment_class=None):
    """Create data generator from image paths"""
    def generator():
        indices = np.arange(len(paths))
        while True:
            np.random.shuffle(indices)
            for start in range(0, len(paths), batch_size):
                end = min(start + batch_size, len(paths))
                batch_indices = indices[start:end]
                
                batch_images = []
                batch_labels = []
                
                for idx in batch_indices:
                    img_path = paths[idx]
                    label = labels[idx]
                    
                    img = tf.keras.preprocessing.image.load_img(
                        img_path, target_size=(IMG_SIZE, IMG_SIZE)
                    )
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    
                    # Apply augmentation only to specified class
                    if augment_class is None or label == augment_class:
                        img_array = datagen.random_transform(img_array)
                    
                    img_array = img_array / 255.0
                    batch_images.append(img_array)
                    batch_labels.append(label)
                
                yield np.array(batch_images), np.array(batch_labels)
    
    return generator

# Create generators
train_generator = create_generator(
    X_train, y_train, train_detector_datagen, BATCH_SIZE, augment_class=1
)

val_generator = create_generator(
    X_val, y_val, val_test_datagen, BATCH_SIZE
)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_generator(
    val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).prefetch(tf.data.AUTOTUNE)

# Calculate steps per epoch
steps_per_epoch = len(X_train) // BATCH_SIZE
validation_steps = len(X_val) // BATCH_SIZE

print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

In [ ]:
# Build model with MobileNetV2 transfer learning

# Load pre-trained MobileNetV2 (without top classification layer)
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze base model layers (transfer learning)
base_model.trainable = False

# Build model
model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Calculate class weights to handle imbalance
total_samples = len(y_train)
n_class_0 = np.sum(y_train == 0)
n_class_1 = np.sum(y_train == 1)

weight_0 = total_samples / (2 * n_class_0)
weight_1 = total_samples / (2 * n_class_1)

class_weights = {0: weight_0, 1: weight_1}

print(f"\nClass weights:")
print(f"  No detector (class 0): {weight_0:.4f}")
print(f"  Detector (class 1): {weight_1:.4f}")

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Model summary
model.summary()

print(f"\nTotal parameters: {model.count_params():,}")
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")

In [ ]:
# Train model

# Callbacks
callbacks = [
    # Early stopping - stop if validation loss doesn't improve for 5 epochs
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate if validation loss plateaus
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    # Save best model during training
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_best.h5'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    
    # TensorBoard logging
    keras.callbacks.TensorBoard(
        log_dir=os.path.join(OUTPUT_DIR, 'logs'),
        histogram_freq=1
    )
]

print("Starting training...")
print("="*60)

# Train model
history = model.fit(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_dataset,
    validation_steps=validation_steps,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("Training completed!")

In [ ]:
# Evaluate and save model

# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history.history['accuracy'], label='Train')
axes[0, 0].plot(history.history['val_accuracy'], label='Validation')
axes[0, 0].set_title('Model Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train')
axes[0, 1].plot(history.history['val_loss'], label='Validation')
axes[0, 1].set_title('Model Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train')
axes[1, 0].plot(history.history['val_precision'], label='Validation')
axes[1, 0].set_title('Model Precision')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train')
axes[1, 1].plot(history.history['val_recall'], label='Validation')
axes[1, 1].set_title('Model Recall')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_history.png'), dpi=300)
plt.show()

# Evaluate on test set
print("\nEvaluating on test set...")
test_generator = create_generator(X_test, y_test, val_test_datagen, BATCH_SIZE)
test_dataset = tf.data.Dataset.from_generator(
    test_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
).prefetch(tf.data.AUTOTUNE)

test_steps = len(X_test) // BATCH_SIZE
test_results = model.evaluate(test_dataset, steps=test_steps, verbose=1)

print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Loss: {test_results[0]:.4f}")
print(f"Accuracy: {test_results[1]:.4f}")
print(f"Precision: {test_results[2]:.4f}")
print(f"Recall: {test_results[3]:.4f}")
print("="*60)

# Save final model in multiple formats
print("\nSaving model...")

# Save as .h5 (single file)
h5_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_final.h5')
model.save(h5_path)
print(f"✓ Saved as .h5: {h5_path}")

# Save as SavedModel (folder)
savedmodel_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_savedmodel')
model.save(savedmodel_path)
print(f"✓ Saved as SavedModel: {savedmodel_path}")

# Save model architecture as JSON
model_json = model.to_json()
json_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_architecture.json')
with open(json_path, 'w') as f:
    f.write(model_json)
print(f"✓ Saved architecture: {json_path}")

# Save training configuration
config = {
    'model_name': MODEL_NAME,
    'img_size': IMG_SIZE,
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'learning_rate': LEARNING_RATE,
    'test_accuracy': float(test_results[1]),
    'test_precision': float(test_results[2]),
    'test_recall': float(test_results[3]),
    'total_images': len(all_data),
    'train_images': len(X_train),
    'val_images': len(X_val),
    'test_images': len(X_test),
    'class_weights': {0: float(weight_0), 1: float(weight_1)},
    'no_detector_count': len(no_detector_data),
    'detector_count': len(detector_data)
}

config_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_config.json')
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)
print(f"✓ Saved config: {config_path}")

print("\n" + "="*60)
print("ALL FILES SAVED SUCCESSFULLY")
print("="*60)
print(f"\nOutput directory: {OUTPUT_DIR}")
print(f"\nFiles created:")
print(f"  - {MODEL_NAME}_final.h5 (model weights)")
print(f"  - {MODEL_NAME}_savedmodel/ (TensorFlow SavedModel)")
print(f"  - {MODEL_NAME}_best.h5 (best model during training)")
print(f"  - {MODEL_NAME}_architecture.json (model structure)")
print(f"  - {MODEL_NAME}_config.json (training configuration)")
print(f"  - training_history.png (training plots)")
print(f"  - logs/ (TensorBoard logs)")
print("="*60)